In [2]:
import os
import findspark
import sys

# os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-21"
os.environ["JAVA_HOME"] = "D:\\anmv2\\Environment\\Jdk1.8"
os.environ["SPARK_HOME"] = "D:\\anmv2\\Environment\\Spark\\spark-3.5.3-bin-hadoop3"
# os.environ["HADOOP_HOME"] = "D:\\anmv2\\Environment\\Hadoop\\hadoop"

# Append pyspark  to Python Path
# sys.path.append("D:\\anmv2\\Environment\\Spark\\spark-3.5.3-bin-hadoop3\\python")
# sys.path.append("D:\\anmv2\\Environment\\Spark\\spark-3.5.3-bin-hadoop3\\python\\lib\\py4j-0.10.9.7-src")

findspark.init()

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

# Tạo SparkConf và cấu hình các tham số
conf = SparkConf() \
    .setMaster('local') \
    .setAppName('Lab2.1 Average Friends') 

# Tạo SparkSession từ SparkConf
spark = SparkSession.builder.config(conf=conf).getOrCreate()

spark.sparkContext.setLogLevel("DEBUG")

# Lấy SparkContext từ SparkSession
sc = spark.sparkContext

In [ ]:
DATA_IN = 'data_lab_2.1_average_friends.csv'

# method 1
# df = spark.read.csv(DATA_IN, header=True, inferSchema=True)
# Chuyển DataFrame thành RDD
# rdd = df.rdd

# method 2
rdd = sc.textFile(DATA_IN)

# remove first line as header
headers = rdd.first()
rdd = rdd.filter(lambda x: x!=headers) 
rdd.collect()

In [22]:
# Kiểm tra vài dòng dữ liệu của RDD
print(rdd.take(5))

['1,Erroll Acom,31,3', '2,Bartel Mosen,25,3', '3,Wells Laybourn,43,1', '4,Marcella Mawer,40,5', '5,Ursa Batterson,25,3']


In [23]:
# Hàm để thực hiện map 
def parseLine(line):
    age = int(line.split(',')[2])
    num_friends = int(line.split(',')[3])
    return (age, num_friends)

In [24]:
# Thực hiện chuyển RDD này về một danh sách các Key-Value với Key là tuổi và Value là số lượng bạn bè. 
# Ví dụ 0 Will 33 385 sẽ được chuyển thành (33, 385) 
rdd_kv = rdd.map(lambda x: parseLine(x))

In [25]:
print(rdd_kv.take(3))

[(31, 3), (25, 3), (43, 1)]


In [33]:
# Sử dụng hàm combineByKey để vừa tính số lượng bạn bè và số lượng số người có cùng tuổi để tính trung bình
sum_count_rdd = rdd_kv.combineByKey(
    lambda value: (value, 1),  # Khởi tạo (value, 1) cho mỗi giá trị đầu tiên của key
    lambda acc, value: (acc[0] + value, acc[1] + 1),  # Cộng dồn value và tăng số lượng cho mỗi key
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # Kết hợp các tổng và số lượng từ các partition khác nhau
)

In [34]:
sum_count_rdd.collect()

[(31, (148, 39)),
 (25, (172, 48)),
 (43, (150, 40)),
 (40, (127, 36)),
 (44, (151, 42)),
 (39, (79, 26)),
 (26, (156, 49)),
 (28, (112, 32)),
 (50, (188, 52)),
 (42, (149, 45)),
 (34, (168, 50)),
 (46, (100, 30)),
 (27, (136, 38)),
 (47, (114, 30)),
 (29, (144, 43)),
 (30, (123, 35)),
 (49, (158, 41)),
 (45, (143, 42)),
 (33, (139, 37)),
 (37, (155, 42)),
 (35, (139, 39)),
 (36, (108, 30)),
 (32, (126, 38)),
 (41, (172, 41)),
 (38, (123, 32)),
 (48, (79, 23))]

#### Guide về hàm combineByKey:
- **lambda value:** (value, 1): Khởi tạo mỗi cặp (value, 1) cho mỗi key.
- **lambda acc, value:** (acc[0] + value, acc[1] + 1): Cộng dồn giá trị value và tăng biến đếm số lượng lên 1 cho từng key.
- **lambda acc1, acc2:** (acc1[0] + acc2[0], acc1[1] + acc2[1]): Kết hợp các giá trị value và số lượng từ các partition khác nhau.
---

```
rdd.combineByKey(createCombiner, mergeValue, mergeCombiners)
```
- ***createCombiner:*** Một hàm áp dụng cho mỗi giá trị value ban đầu để khởi tạo combiners (trình kết hợp) trên mỗi key.
- ***mergeValue:*** Một hàm nhận đối số là combiner và value, sau đó trả về combiner mới sau khi kết hợp value vào.
- ***mergeCombiners:*** Một hàm nhận đối số là hai combiner và kết hợp chúng thành một combiner duy nhất.

In [41]:
# Sử dụng mapValues để tính giá trị trung bình theo keys+
totalsByAge = sum_count_rdd \
                .mapValues(lambda x: round(x[0] / x[1], 3)) \
                .sortByKey() \
                .collect()

In [42]:
print(totalsByAge)

[(25, 3.583), (26, 3.184), (27, 3.579), (28, 3.5), (29, 3.349), (30, 3.514), (31, 3.795), (32, 3.316), (33, 3.757), (34, 3.36), (35, 3.564), (36, 3.6), (37, 3.69), (38, 3.844), (39, 3.038), (40, 3.528), (41, 4.195), (42, 3.311), (43, 3.75), (44, 3.595), (45, 3.405), (46, 3.333), (47, 3.8), (48, 3.435), (49, 3.854), (50, 3.615)]
